In [1]:
controller_file_path="/mnt/Personal/Projects/Hexapod/Transmitter/controller.txt"

In [2]:
# Right Joystick (x axis) (y axis) (button) -- (+Axis 2) (- Axis 3) (Button 14)
# Left Joystick (x axis) (y axis) (button) -- (+Axis 0) (-Axis 1) (Button 13)
# Dpad (up) (right) (down) (left) -- (0,1) (1,0) (0,-1) (-1,0) OR (+DY) (+DX) (-DY) (-DX)
# X Y A B -- (Button 3) (Button 4) (Button 0) (Button 1)
# RB RT -- (Button 7) (+Axis 4)
# LB LT -- (Button 6) (+Axis 5)
# Select Start -- (Button 10) (Button 11) 

In [3]:
def change_key(d, old_key, new_key):
    if old_key in d:
        d[new_key] = d.pop(old_key)
    else:
        print(f"Key '{old_key}' not found.")
        
    return d

def delete_key(d, key):
    d.pop(key)
    
    return d

def value_to_bool(d,key,val=0.5):
    d[key]=  True if d[key]>val else False
    return d

def change_dict_values(control):
    control["A1"]=-control["A1"]
    control["A3"]=-control["A3"]
    
    control=change_key(control,"A2","RJX")
    control=change_key(control,"A3","RJY")
    
    control=change_key(control,"A0","LJX")
    control=change_key(control,"A1","LJY")
    
    control=change_key(control,"B14","RJB")
    control=change_key(control,"B13","LJB")
    
    control=change_key(control,"D0X","DX")
    control=change_key(control,"D0Y","DY")
    
    control=change_key(control,"B3","X")
    control=change_key(control,"B4","Y")
    control=change_key(control,"B0","A")
    control=change_key(control,"B1","B")
    
    control=change_key(control,"B7","RB")
    control=change_key(control,"B6","LB")
    control=change_key(control,"A4","RT")
    control=change_key(control,"A5","LT")
    
    control=change_key(control,"B10","Select")
    control=change_key(control,"B11","Start")
    
    ##############
    #############
    
    
    control = value_to_bool(control,"RT",val=-0.5)
    control = value_to_bool(control,"LT",val=-0.5)
    
    control["DU"]=True if control["DY"]>0 else False
    control["DD"]=True if control["DY"]<0 else False
    control["DR"]=True if control["DX"]>0 else False
    control["DL"]=True if control["DX"]<0 else False
    
    control = delete_key(control,"DX")
    control = delete_key(control,"DY")
    
    control = delete_key(control,"B2")
    control = delete_key(control,"B5")
    control = delete_key(control,"B8")
    control = delete_key(control,"B9")
    control = delete_key(control,"B12")
    
    return control

In [4]:
def write_dict_to_file(filepath, data_dict):
    """
    Deletes the file contents and writes the dictionary as key=value pairs
    line by line.
    """
    with open(filepath, 'w') as f:
        count=0
        for key, value in data_dict.items():
            f.write(f"{key}={value}\n")
            count+=1


In [5]:
class Default_Var:
    def __init__(self):
        self.start_pressed = False
        self.select_pressed = False
        self.lb_pressed = False
        self.rb_pressed = False
        self.rjb_pressed = False
        self.ljb_pressed = False
        self.coxa_counter = 0  # 6 means none
        self.femur_counter = 0 # 6 means none
        self.gait_A_state = False
        self.gait_B_state = False
        self.global_reverse = 0
        self.global_rotate = 0
        

dv = Default_Var()


In [ ]:
def create_code(controller):
    # temp constants
    temp_coxa_femur_angle = 1
    temp_rotate_angle = 1
    temp_height_angle = 1
    temp_roll_angle = 1
    temp_pitch_angle = 1
    temp_yaw_angle = 1

    packet_args = [0] * 25

    # start + select
    if not controller['Select']:
        dv.select_pressed = False
        
    if not controller['Start']:
        dv.start_pressed = False
        
    if controller['Select'] and controller['Start'] and not dv.start_pressed and not dv.select_pressed:
        packet_args[0] = 1
        dv.start_pressed = True
        dv.select_pressed = True

    # LB increment
    if not controller['LB']:
        dv.lb_pressed = False
    if controller['LB'] and not dv.lb_pressed and not controller['RB']:
        dv.coxa_counter = (dv.coxa_counter + 1) % 7
        dv.lb_pressed = True

    # RB increment
    if not controller['RB']:
        dv.rb_pressed = False
    if controller['RB'] and not dv.rb_pressed and not controller['LB']:
        dv.femur_counter = (dv.femur_counter + 1) % 7
        dv.rb_pressed = True

    # both pressed for angle
    if controller['LB'] and controller['RB']:
        if not ((dv.femur_counter == 6 and dv.coxa_counter == 6) or (dv.femur_counter != 6 and dv.coxa_counter != 6)):
        
            angle = 0
            angle = temp_coxa_femur_angle if controller['RJY'] > 0.5 else angle;
            angle = -temp_coxa_femur_angle if controller['RJY'] < -0.5 else angle;
            
            if dv.femur_counter != 6:
                packet_args[1 + dv.coxa_counter] = angle
            elif dv.coxa_counter != 6:
                packet_args[1 + dv.femur_counter + 6] = angle

    # RJB
    if not controller['RJB']:
        dv.rjb_pressed = False
        
    if controller['RJB'] and not dv.rjb_pressed and not controller['LJB']:
        if dv.gait_A_state:
            packet_args[14] = 1
        else:
            packet_args[13] = 1
        dv.gait_A_state = not dv.gait_A_state
        dv.rjb_pressed = True

    # LJB
    if not controller['LJB']:
        dv.ljb_pressed = False
        
    if controller['LJB'] and not dv.ljb_pressed and not controller['RJB']:
        if dv.gait_B_state:
            packet_args[14] = 2
        else:
            packet_args[13] = 2
        dv.gait_B_state = not dv.gait_B_state
        dv.ljb_pressed = True

    # global movement
    if controller['DU']:
        dv.global_reverse = 0
    if controller['DD']:
        dv.global_reverse = 1
        
    packet_args[15] = dv.global_reverse

    # global rotate
    if controller['DL']:
        dv.global_rotate = 0
    if controller['DR']:
        dv.global_rotate = 1
        
    ## DL and DR not needed - can be used for speed
    
    packet_args[16] = dv.global_rotate

    # move_base
    if controller['RT']:
        packet_args[17] = 1
        
    ## LT unused

    # rotate_base
    if controller['RJX'] != 0:
        
        angle = 0
        angle = temp_rotate_angle if controller['RJX'] > 0.5 else angle;
        angle = -temp_rotate_angle if controller['RJX'] < -0.5 else angle;
        
        packet_args[18] = angle

    # base_height
    if controller['LJY'] != 0:
        
        angle = 0
        angle = temp_height_angle if controller['LJY'] > 0.5 else angle;
        angle = -temp_height_angle if controller['LJY'] < -0.5 else angle;
        
        packet_args[19] = angle

    # base_yaw
    if controller['LJX'] != 0:
        
        angle = 0
        angle = temp_yaw_angle if controller['LJX'] > 0.5 else angle;
        angle = -temp_yaw_angle if controller['LJX'] < -0.5 else angle;
        
        packet_args[20] = angle

    # base_roll
    if controller['X']:
        packet_args[21] = -temp_roll_angle
    if controller['B']:
        packet_args[21] = temp_roll_angle

    # base_pitch
    if controller['A']:
        packet_args[22] = -temp_pitch_angle
    if controller['Y']:
        packet_args[22] = temp_pitch_angle

    # org_reset
    if controller['Start'] and not controller['Select']:
        packet_args[23] = 1

    # default_reset
    if controller['Select'] and not controller['Start']:
        packet_args[24] = 1

    return packet_args


In [ ]:
import serial
import time

# global serial object
ser = None

def serial_setup(port, baudrate):
    global ser
    ser = serial.Serial(port, baudrate, timeout=1)
    time.sleep(5)  # let Arduino reset
    print("Serial initialized")

def serial_transfer(data_array):
    
    # array of int8_t[25]
    
    global ser
    if ser is None:
        raise RuntimeError("Serial not initialized. Call serial_setup() first.")

    byte_data = bytearray([(x + 256) % 256 for x in data_array])  # handle signed
    ser.write(byte_data)
    
    print("Sent : "+str(time.time()))
    # time.sleep(0.2)  # adjust as needed
        



In [ ]:
import pygame
from IPython.display import clear_output
import time

serial_setup("/dev/ttyACM0", 9600)

# initialize pygame
pygame.init()

# initialize the joystick
pygame.joystick.init()

# check for connected joysticks
if pygame.joystick.get_count() == 0:
    print("No joystick found")
    exit()
else:
    joystick = pygame.joystick.Joystick(0)
    joystick.init()
    print(f"Detected controller: {joystick.get_name()}")

# event loop
try:
    while True:
        start_time = time.time()
        
        pygame.event.pump()  # updates internal event queue
        clear_output(wait=True)
        
        controller_text={}

        # read axes
        for i in range(joystick.get_numaxes()):
            axis = joystick.get_axis(i)
            controller_text["A"+str(i)]=axis
            # print(f"Axis {i}: {axis:.3f}", end="  ")
        # print()
        # read buttons
        for i in range(joystick.get_numbuttons()):
            button = joystick.get_button(i)
            button = True if button>0.5 else False
            controller_text["B"+str(i)]=button
            # print(f"But {i}: {button}", end="  ")
            
        # print()
            
        # read hats (d-pad)
        for i in range(joystick.get_numhats()):
            hat = joystick.get_hat(i)
            controller_text["D"+str(i)+"X"]=hat[0]
            controller_text["D"+str(i)+"Y"]=hat[1]
            # print(f"Hat {i}: {hat}", end="  ")

        # print()
        # print(controller_text)
        controller_text=change_dict_values(controller_text)
        
        data_packet = create_code(controller_text)
        
        serial_transfer(data_packet)
        
        # write_dict_to_file(controller_file_path,controller_text)
        
        # print()
        
        total_wait = 0.2
        
        delta_time=time.time()-start_time
        
        if(total_wait>delta_time):
            time.sleep(total_wait-delta_time)
        
except KeyboardInterrupt:
    print("Stopped")

finally:
    pygame.quit()


Axis 0: 0.004  Axis 1: -0.004  Axis 2: 0.004  Axis 3: -0.004  Axis 4: -1.000  Axis 5: -1.000  
But 0: False  But 1: False  But 2: False  But 3: False  But 4: False  But 5: False  But 6: False  But 7: False  But 8: False  But 9: False  But 10: False  But 11: False  But 12: False  But 13: False  But 14: False  
Hat 0: (0, 0)  


: 